In [1]:
!pip install transformers datasets 'accelerate>=0.26.0' sentencepiece -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 267.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 142.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 139.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 261.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 175.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 252.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 153.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9


In [2]:
import pandas as pd

# eval_df = pd.read_csv("eval_data.csv")
eval_df = pd.read_csv("data_llm_eval.csv")
eval_df = eval_df.dropna()

In [3]:
eval_df.head()

,id,joke,score,llama_score,llama_reason,gpt_score,gpt_reason
0,1,What do you call a mushroom that buys all the ...,0.427833,7,"The joke is a play on words, using the scienti...",5.0,The joke relies on a classic pun (fungi/fun‑gu...
1,2,What did the digital clock say to the grandfat...,0.376117,8,"The joke is a play on words, using the phrase ...",5.0,The joke is a simple pun about a digital clock...
2,3,I'm in the terminator musical. I'll be Bach.,0.314998,8,"This joke is a play on words, combining the co...",6.0,The joke is a short pun that mixes the Termina...
3,4,Will you tell you the story of the huge sad wa...,0.404325,8,"This joke is a play on words, using the phrase...",5.0,The joke relies on a simple pun—'you'll never ...
4,5,What do you call a cat in an iron man suit ? I...,0.319699,6,"The joke is a play on words, combining 'Iron M...",5.0,The joke is a straightforward pun (Iron Meow) ...


In [4]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, TrainingArguments, Trainer

eval_df["labels"] = (eval_df["llama_score"] + eval_df["gpt_score"]) // 2
eval_df["labels"] = eval_df["labels"] = eval_df["labels"].replace(-1, 0)
eval_df.labels = eval_df.labels.astype(int)
eval_df.joke = eval_df.joke.astype(str)
eval_df = eval_df[["joke", "labels"]]

In [5]:
synthetic_df = pd.read_csv("eval_data.csv")
synthetic_df["labels"] = synthetic_df.score
synthetic_df = synthetic_df[["joke", "labels"]]

In [6]:
eval_df = pd.concat([eval_df, synthetic_df])

In [7]:
eval_df.head()

,joke,labels
0,What do you call a mushroom that buys all the ...,6
1,What did the digital clock say to the grandfat...,6
2,I'm in the terminator musical. I'll be Bach.,7
3,Will you tell you the story of the huge sad wa...,6
4,What do you call a cat in an iron man suit ? I...,5


In [9]:
train_df, test_df = train_test_split(eval_df, test_size=0.2, random_state=42, stratify=eval_df.labels)
test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42, stratify=test_df.labels)
print(len(train_df))
print(len(test_df))
print(len(val_df))
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)
val_ds = Dataset.from_pandas(val_df)

42083
5260
5261


In [10]:
model_name = "FacebookAI/xlm-roberta-large"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["joke"], truncation=True, padding="max_length", max_length=128)

train_ds = train_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

Map:   0%|          | 0/42083 [00:00<?, ? examples/s]

Map:   0%|          | 0/5260 [00:00<?, ? examples/s]

Map:   0%|          | 0/5261 [00:00<?, ? examples/s]

In [11]:
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    rmse = mean_squared_error(labels, preds)
    mae = mean_absolute_error(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"rmse": rmse, "accuracy": acc, "mae": mae}

In [12]:
labels = np.array(train_ds["labels"])

In [13]:
import numpy as np
import torch

binary_counts = np.array([(labels == 0).sum(), (labels != 0).sum()], dtype=float)

binary_weights = (binary_counts.sum() / (2 * binary_counts))
binary_weights = torch.tensor(binary_weights, dtype=torch.float)

child_labels = labels[labels != 0]    # keep only 1–10
child_counts = np.array([(child_labels == c).sum() for c in range(1, train_df.labels.nunique())], dtype=float)

child_weights = (child_counts.sum() / (10 * child_counts))
child_weights = torch.tensor(child_weights, dtype=torch.float)

In [14]:
import torch
import torch.nn as nn
from transformers import XLMRobertaPreTrainedModel, XLMRobertaModel

class HierarchicalClassifier(XLMRobertaPreTrainedModel):
    
    def __init__(self, config, num_child_labels=10, class_weights_binary=None, class_weights_child=None):
        h = config.hidden_size
        super().__init__(config)
        
        self.roberta = XLMRobertaModel(config)
        self.dropout = nn.Dropout(0.5)
        self.proj = nn.Linear(h, h//2)
        self.relu = nn.ReLU()
        self.binary_head = nn.Linear(h//2, 2)
        self.child_head = nn.Linear(h//2, num_child_labels)
        self.loss_bin = nn.CrossEntropyLoss(weight=class_weights_binary)
        self.loss_child = nn.CrossEntropyLoss(weight=class_weights_child)
        
        self.post_init()

    def forward(self, input_ids, attention_mask=None, labels=None):
        a = torch.tensor(0.5, requires_grad=True)
        b = torch.tensor(0.5, requires_grad=True)
        loss = None
        
        x = self.roberta(input_ids, attention_mask=attention_mask)
        x = x.last_hidden_state[:, 0] # cls token
        x = self.dropout(x)
        x = self.proj(x)
        x = self.relu(x)
    
        lb = self.binary_head(x)
        lc = self.child_head(x)

        pb = torch.softmax(lb, dim=-1)
        pc = torch.softmax(lc, dim=-1)
        sc = pc * pb[:, 1].unsqueeze(-1)
        logits = torch.cat([pb[:, 0].unsqueeze(-1), sc], dim=-1)

        probs = torch.softmax(logits, dim=-1)
        expected = (probs * torch.arange(0, logits.size(1), device=logits.device)).sum(dim=-1)
        reg_loss = torch.mean((expected - labels.float()) ** 2)
        
        if labels is not None:
            bt = (labels != 0).long()
            loss_b = self.loss_bin(lb, bt)
            nz = bt == 1
            if nz.any():
                cl = labels[nz] - 1
                loss_c = self.loss_child(lc[nz], cl)
            else:
                loss_c = 0.0
            clf_loss = loss_b + loss_c

            loss = a * reg_loss + b * clf_loss
        
        return {"loss": loss, "logits": logits}


In [15]:
n_labels = len(train_df.labels.unique())
# model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=n_labels)

model = HierarchicalClassifier.from_pretrained(
    model_name,
    num_child_labels=n_labels-1,
    class_weights_binary=binary_weights,
    class_weights_child=child_weights
)

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of HierarchicalClassifier were not initialized from the model checkpoint at FacebookAI/xlm-roberta-large and are newly initialized: ['binary_head.bias', 'binary_head.weight', 'child_head.bias', 'child_head.weight', 'loss_bin.weight', 'loss_child.weight', 'proj.bias', 'proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
args = TrainingArguments(
    output_dir="./xlm-roberta-joke-rating",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=30,
    learning_rate=2e-5,
    eval_strategy="epoch",
    save_strategy="no",
    fp16=True,
    warmup_ratio=0.1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
test_results = trainer.predict(test_ds)

In [ ]:
test_results